# NEIGHBORING TORONTO CITY

## EVERYTHING IS IN ONE NOTEBOOK ITSELF

### START OF WEEK 3 ASSIGNMENT

#### IMPORT LIBRARIES

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### GET THE DATA OF TORONTO CITY INTO A DATAFRAME

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = pd.read_html(url)[0]
data.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### CLEANING UP THE DATA

In [3]:
Toronto_data = data.where(data['Borough'] != 'Not assigned').dropna()
Toronto_data.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
Toronto_data.sort_values('Postal Code', ascending = True, inplace = True)
Toronto_data = Toronto_data.reset_index().drop('index', axis = 1)
Toronto_data.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
for index, row in Toronto_data.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] == row['Borough']
Toronto_data.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
Toronto_data.shape

(103, 3)

#### TRIED FINDING THE LATITUDE AND LONGITUDE OF THE POSTAL CODE (USING GEOPY)

In [7]:
# Create new columns
Toronto_data['Latitude'] = None
Toronto_data['Longitude'] = None
geolocator = Nominatim(user_agent="to_explorer")

# iterate over every row to find its latitude and logitude
for index, row in Toronto_data.iterrows():
    Neighbourhood = row['Neighbourhood'].split(',')[0]
    print(Neighbourhood)
    address = '{}, Toronto, Ontario'.format(Neighbourhood)
    location = geolocator.geocode(address)
    
    if location is None:
        continue
    latitude = location.latitude
    longitude = location.longitude
    
    # Add it to the new columns
    row['Latitude'] = latitude
    row['Longitude'] = longitude
    
    


Malvern
Rouge Hill
Guildwood
Woburn
Cedarbrae
Scarborough Village
Kennedy Park
Golden Mile
Cliffside
Birch Cliff
Dorset Park
Wexford
Agincourt
Clarks Corners
Milliken
Steeles West
Upper Rouge
Hillcrest Village
Fairview
Bayview Village
York Mills
Willowdale
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor
Northwood Park
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West
India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park
Summerhill West
Rosedale
St. James Town
Church and Wellesley
Regent Park
Garden District
St. James Town
Berczy Park
Central Bay Street
Richmond
Harbourfront East
Toronto Dominion Centre
Commerce Court
Bedford Park
Roselawn
Forest Hill North & West
The Annex
University of Toronto
Kensington Market
CN Tower
Stn A PO Boxes
First Canadian Place
Lawrence Manor
Glencairn
Humewood-Cedarvale
C

#### TRIED FINDING THE LATITUDE AND LONGITUDE OF THE POSTAL CODE (USING GEOCODER)

In [8]:
# import library
import geocoder

# Create new columns
Toronto_data['Latitude'] = None
Toronto_data['Longitude'] = None

# initiatize variable to None
coordinates = None

# iterate over every row to find its latitude and logitude
for index, row in Toronto_data.iterrows():
    Postal_code = row['Postal code']
    # While loop till we get locations
    while (coordinates == None):
        g = geocoder.google('{}, Toronto, Ontario'.format(Postal_code))
        coordinates = g.latlng
    
    latitude = coordinates[0]
    longitude = coordinates[1]
    
    # Add it to the new columns
    row['Latitude'] = latitude
    row['Longitude'] = longitude

KeyError: 'Postal code'

#### USING THE GIVEN COORDINATES FILE 

In [9]:
# Read the csv file containing the coordinates
coordinates_df = pd.read_csv('Geospatial_Coordinates.csv')
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
coordinates_df.shape

(103, 3)

In [11]:
# Adding the Latitude and Longitude values 
Toronto_data['Latitude'] = coordinates_df['Latitude']
Toronto_data['Longitude'] = coordinates_df['Longitude']
Toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### END OF WEEK 3 ASSIGNMENT

### START OF WEEK 4 ASSIGNMENT

#### Problem Statement

You want to open a new coffee shop in Toronto, CA. You want to find the optimal venue to set up your cafe to maximize the profit.

1) We select the top 5 neighborhood having maximum number of offices.<br>
2) We select the top 5 neighborhood having maximum number of colleges.<br>
3) We select the top 5 neighborhood having maximum number of schools.<br>
4) Make a union list of all these neighborhood.<br>
5) For all these neighborhood, we find which one of these has the minimum number of coffee shops.<br>
6) Once we find the neighborhood where we want to set up our shop, we search for trending venues in the neighborhood and set up a shop there.<br>
7) If there are no trending venues, visualize the colleges and schools in the area and select the area where attributes are conjusted.<br>
8) If not happy with the result, select the second neighborhood with minimum number of coffee shops.<br>

We will be using the Toranto_data that we created earlier to get the neighbouhood data and the Foursquare API for location data

### END OF WEEK 4 ASSIGNMENT